In [2]:
!pip install gradio
!pip install transformers datasets accelerate
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 77.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 97.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have goog

In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
output_dir = "/kaggle/input/code-llama-sql/transformers/default/1/sql-code-llama/checkpoint-400"
model = PeftModel.from_pretrained(model, output_dir)

/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [6]:
# Define the inference function
def generate_sql(context, question):
    # Construct the prompt
    eval_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

    You must output the SQL query that answers the question.
    ### Input:
    {question}

    ### Context:
    {context}

    ### Response:
    """
    # Tokenize and generate response
    model_input = tokenizer(eval_prompt, return_tensors="pt").to(next(model.parameters()).device)
    model.eval()
    with torch.no_grad():
        output = model.generate(**model_input, max_new_tokens=100)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the SQL query from the response
    response_marker = "### Response:"
    response_start = response.find(response_marker)
    if response_start != -1:
        response_text = response[response_start + len(response_marker):].strip()
        return response_text
    else:
        return "No valid SQL query generated."

In [7]:
# Create the Gradio interface
interface = gr.Interface(
  fn=generate_sql,
  inputs=[
      gr.Textbox(lines=10, placeholder="Enter the database schema here...", label="Context"),
      gr.Textbox(lines=2, placeholder="Enter your question here...", label="Question")
  ],
  outputs="text",
  examples=[
      [
          "CREATE TABLE employees (id INT, name VARCHAR, salary DECIMAL, department_id INT);",
          "Which employees earn more than \$60,000?"
      ],
      [
          "CREATE TABLE customers (id INT, name VARCHAR, city VARCHAR, email VARCHAR);",
          "What are the names of customers who live in New York?"
      ],
      [
          "CREATE TABLE orders (order_id INT, order_date DATE, customer_id INT, total_amount DECIMAL);",
          "Which orders were placed after January 1, 2023?"
      ]
  ],
  title="Text-to-SQL Model Interface",
  description="Enter the database schema in the 'Context' field and your question in the 'Question' field. The model will generate the corresponding SQL query."
)

# Launch the interface
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1818c03bbb24431a52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
